In [1]:
import warnings
warnings.filterwarnings("ignore")
from copy import deepcopy
from datetime import datetime
from pathlib import Path

import numpy as np
import pandas as pd
import sys

sys.path.append("/code")

from tqdm import tqdm
import torch
# device = torch.device('cpu')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
# import gym
# import recogym

import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim

torch.backends.cudnn.benchmark = torch.cuda.is_available()
if torch.cuda.is_available():
    torch.set_float32_matmul_precision("high")  # TF32 = big speedup on Ada


from sklearn.utils import check_random_state

# implementing OPE of the IPWLearner using synthetic bandit data
from sklearn.linear_model import LogisticRegression

import matplotlib.pyplot as plt

from scipy.special import softmax
import optuna
# from memory_profiler import profile


from estimators import (
    DirectMethod as DM
)

from simulation_utils import (
    eval_policy,
    generate_dataset,
    create_simulation_data_from_pi,
    get_train_data,
    get_opl_results_dict,
    CustomCFDataset,
    calc_reward,
    get_weights_info
)

from models import (    
    LinearCFModel,
    NeighborhoodModel,
    BPRModel, 
    RegressionModel
)

from training_utils import (
    train,
    validation_loop, 
    cv_score_model
 )

from custom_losses import (
    SNDRPolicyLoss,
    IPWPolicyLoss
    )

random_state=12345
random_ = check_random_state(random_state)

pd.options.display.float_format = '{:,.8f}'.format

Using device: cpu
Using device: cpu
Using device: cpu


In [2]:
def get_trial_results(
    our_x, 
    our_a, 
    emb_x, 
    emb_a, 
    original_x, 
    original_a, 
    dataset, 
    val_data, 
    original_policy_prob, 
    neighberhoodmodel, 
    regression_model, 
    dm
):
    policy = np.expand_dims(softmax(our_x @ our_a.T, axis=1), -1)
    policy_reward = calc_reward(dataset, policy)
    eval_metrics = eval_policy(neighberhoodmodel, val_data, original_policy_prob, policy)
    action_diff_to_real = np.sqrt(np.mean((emb_a - our_a) ** 2))
    action_delta = np.sqrt(np.mean((original_a - our_a) ** 2))
    context_diff_to_real = np.sqrt(np.mean((emb_x - our_x) ** 2))
    context_delta = np.sqrt(np.mean((original_x - our_x) ** 2))

    row = np.concatenate([
        np.atleast_1d(policy_reward),
        np.atleast_1d(eval_metrics),
        np.atleast_1d(action_diff_to_real),
        np.atleast_1d(action_delta),
        np.atleast_1d(context_diff_to_real),
        np.atleast_1d(context_delta)
    ])
    reg_dm = dm.estimate_policy_value(policy[val_data['x_idx']], regression_model.predict(val_data['x']))
    reg_results = np.array([reg_dm])
    conv_results = np.array([row])
    return get_opl_results_dict(reg_results, conv_results)

## `trainer_trial` Function

This function runs policy learning experiments using offline bandit data and evaluates various estimators.

### Parameters
- **num_runs** (int): Number of experimental runs per training size
- **num_neighbors** (int): Number of neighbors to consider in the neighborhood model
- **num_rounds_list** (list): List of training set sizes to evaluate
- **dataset** (dict): Contains dataset information including embeddings, action probabilities, and reward probabilities
- **batch_size** (int): Batch size for training the policy model
- **num_epochs** (int): Number of training epochs for each experiment
- **lr** (float, default=0.001): Learning rate for the optimizer

### Process Flow
1. Initializes result structures and retrieval models
2. For each training size in `num_rounds_list`:
   - Creates a uniform logging policy and simulates data
   - Generates training data for offline learning
   - Fits regression and neighborhood models for reward estimation
   - Initializes and trains a counterfactual policy model
   - Evaluates policy performance using various estimators
   - Collects metrics on policy reward and embedding quality

### Returns
- **DataFrame**: Results table with rows indexed by training size and columns for various metrics:
  - `policy_rewards`: True expected reward of the learned policy
  - Various estimator errors (`ipw`, `reg_dm`, `conv_dm`, `conv_dr`, `conv_sndr`)
  - Variance metrics for each estimator
  - Embedding quality metrics comparing learned representations to ground truth

### Implementation Notes
- Uses uniform random logging policy for collecting offline data
- Employs Self-Normalized Doubly Robust (SNDR) policy learning
- Measures embedding quality via RMSE to original/ground truth embeddings

In [3]:
def trainer_trial(
    num_runs,
    num_neighbors,
    train_sizes,
    dataset,
    batch_size,
    val_size=2000,
    n_trials=10,    
    prev_best_params=None
):

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    torch.backends.cudnn.benchmark = torch.cuda.is_available()
    if torch.cuda.is_available():
        torch.set_float32_matmul_precision("high")

    dm = DM()
    results = {}

    our_x, our_a = dataset["our_x"], dataset["our_a"]
    emb_x, emb_a = dataset["emb_x"], dataset["emb_a"]

    original_x, original_a = dataset["original_x"], dataset["original_a"]
    n_users, n_actions, emb_dim = dataset["n_users"], dataset["n_actions"], dataset["emb_dim"]

    all_user_indices = np.arange(n_users, dtype=np.int64)

    def T(x):
        return torch.as_tensor(x, device=device, dtype=torch.float32)

    def _mean_dict(dicts):
        """
        Robust mean over a list of dicts with numeric/scalar/1D-array values.
        Returns a single dict with elementwise means.
        """
        if not dicts:
            return {}
        keys = dicts[0].keys()
        out = {}
        for k in keys:
            vals = [d[k] for d in dicts if k in d]
            # try to convert each to np.array and average
            arrs = [np.asarray(v) for v in vals]
            # broadcast to same shape if scalars/1D
            stacked = np.stack(arrs, axis=0)
            out[k] = np.mean(stacked, axis=0)
        return out

    # ===== unpack dataset (keep originals safe) =====
    our_x_orig, our_a_orig = our_x, our_a
    emb_x, emb_a = emb_x, emb_a
    original_x, original_a = original_x, original_a
    n_users, n_actions, emb_dim = n_users, n_actions, emb_dim
    all_user_indices = np.arange(n_users, dtype=np.int64)

    dm = DM()
    results = {}
    best_hyperparams_by_size = {}
    last_best_params = prev_best_params if prev_best_params is not None else None

    # ===== baseline (sample size = 0) using get_trial_results =====
    pi_0 = softmax(our_x_orig @ our_a_orig.T, axis=1)
    original_policy_prob = np.expand_dims(pi_0, -1)

    simulation_data = create_simulation_data_from_pi(
        dataset, pi_0, val_size, random_state=0
    )

    # use same data for train/val just to generate the baseline row
    train_data = get_train_data(n_actions, val_size, simulation_data, np.arange(val_size), our_x_orig)
    val_data   = get_train_data(n_actions, val_size, simulation_data, np.arange(val_size), our_x_orig)

    regression_model = RegressionModel(
        n_actions=n_actions, action_context=our_x_orig,
        base_model=LogisticRegression(random_state=12345)
    )

    regression_model.fit(train_data['x'], train_data['a'], train_data['r'])

    neighberhoodmodel = NeighborhoodModel(
        train_data['x_idx'], train_data['a'],
        our_a_orig, our_x_orig, train_data['r'],
        num_neighbors=num_neighbors
    )

    # baseline row produced via get_trial_results
    results[0] = get_trial_results(
        our_x_orig, our_a_orig, emb_x, emb_a, original_x, original_a,
        dataset, val_data, original_policy_prob,
        neighberhoodmodel, regression_model, dm
    )

    # ===== main loop over training sizes =====
    for train_size in train_sizes:

        # we’ll collect per-run trial dicts generated by get_trial_results
        trial_dicts_this_size = []
        best_hyperparams_by_size[train_size] = {}

        # --- prepare a resampling for Optuna’s objective (shared loaders built per-run inside objective) ---
        # We’ll do Optuna per-run (fresh resample + search), then final fit with best params, then get_trial_results.

        for run in range(num_runs):

            # --- resample for this run ---
            pi_0 = softmax(our_x_orig @ our_a_orig.T, axis=1)
            original_policy_prob = np.expand_dims(pi_0, -1)

            simulation_data = create_simulation_data_from_pi(
                dataset, pi_0, train_size + val_size,
                random_state=(run + 1) * (train_size + 17)
            )

            idx_train = np.arange(train_size)
            train_data = get_train_data(n_actions, train_size, simulation_data, idx_train, our_x_orig)
            val_idx   = np.arange(val_size) + train_size
            val_data  = get_train_data(n_actions, val_size, simulation_data, val_idx, our_x_orig)

            num_workers = 4 if torch.cuda.is_available() else 0

            cf_dataset = CustomCFDataset(
                train_data['x_idx'], train_data['a'], train_data['r'], original_policy_prob
            )

            # val_loader = DataLoader(
            #     val_dataset, batch_size=val_size, shuffle=False,
            #     pin_memory=torch.cuda.is_available(),
            #     num_workers=num_workers, persistent_workers=bool(num_workers)
            # )


            # --- Optuna objective bound to this run's data ---
            def objective(trial):
                lr = trial.suggest_float("lr", 1e-4, 1e-1, log=True)
                epochs = trial.suggest_int("num_epochs", 1, 10)
                trial_batch_size = trial.suggest_categorical("batch_size", [64, 128, 256, 512])
                trial_num_neighbors = trial.suggest_int("num_neighbors", 3, 15)
                lr_decay = trial.suggest_float("lr_decay", 0.8, 1.0)

                trial_neigh_model = NeighborhoodModel(
                    train_data['x_idx'], train_data['a'],
                    our_a_orig, our_x_orig, train_data['r'],
                    num_neighbors=trial_num_neighbors
                )

                trial_scores_all = torch.as_tensor(
                    trial_neigh_model.predict(all_user_indices),
                    device=device, dtype=torch.float32
                )

                trial_model = LinearCFModel(
                    n_users, n_actions, emb_dim,
                    initial_user_embeddings=T(our_x_orig),
                    initial_actions_embeddings=T(our_a_orig)
                ).to(device)

                assert (not torch.cuda.is_available()) or next(trial_model.parameters()).is_cuda

                final_train_loader = DataLoader(
                    cf_dataset, batch_size=trial_batch_size, shuffle=True,
                    pin_memory=torch.cuda.is_available(),
                    num_workers=num_workers, persistent_workers=bool(num_workers)
                )

                current_lr = lr
                for epoch in range(epochs):
                    if epoch > 0:
                        current_lr *= lr_decay
                        
                    train(
                        trial_model, final_train_loader, trial_scores_all,
                        criterion=SNDRPolicyLoss(), num_epochs=1, lr=current_lr, device=str(device)
                    )

                trial_x, trial_a = trial_model.get_params()
                trial_x = trial_x.detach().cpu().numpy()
                trial_a = trial_a.detach().cpu().numpy()

                pi_i = softmax(trial_x @ trial_a.T, axis=1)
                train_actions = train_data['a']
                train_users = train_data['x_idx']

                print("Train wi info: {}".format(get_weights_info(pi_i[train_users, train_actions], original_policy_prob[train_users, train_actions])))
                print(f"actual reward: {calc_reward(dataset, np.expand_dims(pi_i, -1))}")

                # print(get_weights_info(pi_i, original_policy_prob))
                # validation reward for selection
                return cv_score_model(val_data, trial_scores_all, pi_i)


            # --- run Optuna for this run ---
            study = optuna.create_study(direction="maximize")
            
            if last_best_params is not None:
                study.enqueue_trial(last_best_params)

            study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

            best_params = study.best_params
            last_best_params = best_params  # optional warm-start to next run
            best_hyperparams_by_size[train_size][run] = {
                "params": best_params,
                "reward": study.best_value
            }


            # --- final training with best params on this run’s data ---
            regression_model = RegressionModel(
                n_actions=n_actions, action_context=our_x_orig,
                base_model=LogisticRegression(random_state=12345)
            )
            regression_model.fit(
                train_data['x'], train_data['a'], train_data['r'],
                original_policy_prob[train_data['x_idx'], train_data['a']].squeeze()
            )

            neighberhoodmodel = NeighborhoodModel(
                train_data['x_idx'], train_data['a'],
                our_a_orig, our_x_orig, train_data['r'],
                num_neighbors=best_params['num_neighbors']
            )
            scores_all = torch.as_tensor(
                neighberhoodmodel.predict(all_user_indices),
                device=device, dtype=torch.float32
            )

            model = LinearCFModel(
                n_users, n_actions, emb_dim,
                initial_user_embeddings=T(our_x_orig),
                initial_actions_embeddings=T(our_a_orig)
            ).to(device)
            assert (not torch.cuda.is_available()) or next(model.parameters()).is_cuda

            train_loader = DataLoader(
                cf_dataset, batch_size=batch_size, shuffle=True,
                pin_memory=torch.cuda.is_available(),
                num_workers=num_workers, persistent_workers=bool(num_workers)
            )

            current_lr = best_params['lr']
            for epoch in range(best_params['num_epochs']):
                if epoch > 0:
                    current_lr *= best_params['lr_decay']
                train(
                    model, train_loader, scores_all,
                    criterion=SNDRPolicyLoss(), num_epochs=1, lr=current_lr, device=str(device)
                )

            # learned embeddings (do NOT overwrite originals)
            learned_x_t, learned_a_t = model.get_params()
            learned_x = learned_x_t.detach().cpu().numpy()
            learned_a = learned_a_t.detach().cpu().numpy()

            # --- produce the per-run result via get_trial_results ---
            trial_res = get_trial_results(
                learned_x, learned_a,          # learned (policy) embeddings
                emb_x, emb_a,                  # ground-truth embedding refs
                original_x, original_a,        # original clean refs
                dataset,
                val_data,                      # use this run's val split
                original_policy_prob,
                neighberhoodmodel,
                regression_model,
                dm
            )

            trial_dicts_this_size.append(trial_res)

            # memory hygiene
            torch.cuda.empty_cache()

        # === aggregate per-run results (mean) and store under this train_size ===
        results[train_size] = _mean_dict(trial_dicts_this_size)

    return pd.DataFrame.from_dict(results, orient='index'), best_hyperparams_by_size

## Learning

We will run several simulations on a generated dataset, the dataset is generated like this:
$$ \text{We have users U and actions A } u_i \sim N(0, I_{emb_dim}) \ a_i \sim N(0, I_{emb_dim})$$
$$ p_{ij} = 1 / (5 + e^{-(u_i.T a_j)}) $$
$$r_{ij} \sim Bin(p_{ij})$$

We have a policy $\pi$
and it's ground truth reward is calculated by
$$R_{gt} = \sum_{i}{\sum_{j}{\pi_{ij} * p_{ij}}} $$

Our parameters for the dataset will be
$$EmbDim = 5$$
$$NumActions= 150$$
$$NumUsers = 150$$
$$NeighborhoodSize = 6$$

to learn a new policy from $\pi$ we will sample from:
$$\pi_{start} = (1-\epsilon)*\pi + \epsilon * \pi_{random}$$

In [4]:
dataset_params = dict(
                    n_actions= 500,
                    n_users = 500,
                    emb_dim = 16,
                    # sigma = 0.1,
                    eps = 0.6, # this is the epsilon for the noise in the ground truth policy representation
                    ctr = 0.1
                    )

train_dataset = generate_dataset(dataset_params)

Random Item CTR: 0.07066414727263938
Optimal greedy CTR: 0.09999926940951757
Optimal Stochastic CTR: 0.09995326955796031
Our Initial CTR: 0.08610747363354625


In [5]:
num_runs = 1
batch_size = 200
num_neighbors = 6
n_trials_for_optuna = 20
# num_rounds_list = [500, 1000, 2000, 10000, 20000]
# num_rounds_list = [500, 1000, 2000]
num_rounds_list = [15000]


# Manually define your best parameters
best_params_to_use = {
    "lr": 0.096,  # Learning rate
    "num_epochs": 5,  # Number of training epochs
    "batch_size": 64,  # Batch size for training
    "num_neighbors": 8,  # Number of neighbors for neighborhood model
    "lr_decay": 0.85  # Learning rate decay factor
}

### 1

$$emb = 0.7 * gt + 0.3 * noise$$
$$lr = 0.005$$
$$n_{epochs} = 1$$
$$BatchSize=50$$

In [6]:
print("Value of num_rounds_list:", num_rounds_list)

# Run the optimization
df4, best_hyperparams_by_size = trainer_trial(num_runs, num_neighbors, num_rounds_list, train_dataset, batch_size, val_size=10000, n_trials=n_trials_for_optuna, prev_best_params=best_params_to_use)

# # Print best hyperparameters for each training size
# print("\n=== BEST HYPERPARAMETERS BY TRAINING SIZE ===")
# for train_size, params in best_hyperparams_by_size.items():
#     print(f"\nTraining Size: {train_size}")
#     # print(f"Best Reward: {params['reward']:.6f}")
#     print("Parameters:")
#     for param_name, value in params['params'].items():
#         print(f"  {param_name}: {value}")
# print("===========================\n")

# Show the performance metrics
df4[['policy_rewards', 'ipw', 'reg_dm', 'conv_dm', 'conv_dr', 'conv_sndr', 'action_diff_to_real', 'action_delta', 'context_diff_to_real', 'context_delta']]

Value of num_rounds_list: [15000]
{'gini': np.float64(0.47165337067628044), 'ess': np.float64(4299.836390949015), 'max_wi': np.float64(39.02207562454071), 'min_wi': np.float64(0.008541729602724986)}


[I 2025-10-27 23:20:50,001] A new study created in memory with name: no-name-f4656db7-3a4d-4f33-9869-96b29d82a165
Best trial: 0. Best value: 0.0719627:   5%|▌         | 1/20 [00:44<14:11, 44.82s/it]

Train wi info: {'gini': np.float64(0.999646163854872), 'ess': np.float64(5.634283506987809), 'max_wi': np.float64(4346.13005127459), 'min_wi': np.float64(0.0)}
actual reward: [0.07844657]
{'gini': np.float64(0.9998023537765777), 'ess': np.float64(2.012897323932942), 'max_wi': np.float64(2038.673415495716), 'min_wi': np.float64(0.0)}
Cross-validated error: 0.006261881785689511
[I 2025-10-27 23:21:34,824] Trial 0 finished with value: 0.07196269215221371 and parameters: {'lr': 0.096, 'num_epochs': 5, 'batch_size': 64, 'num_neighbors': 8, 'lr_decay': 0.85}. Best is trial 0 with value: 0.07196269215221371.


Best trial: 1. Best value: 0.0736587:  10%|█         | 2/20 [01:25<12:41, 42.30s/it]

Train wi info: {'gini': np.float64(0.028013777796080327), 'ess': np.float64(14942.23837489102), 'max_wi': np.float64(1.489931280462149), 'min_wi': np.float64(0.8713135516788827)}
actual reward: [0.08612014]
{'gini': np.float64(0.02875486793061097), 'ess': np.float64(9960.921018362127), 'max_wi': np.float64(1.497317319184339), 'min_wi': np.float64(0.8880170187116933)}
Cross-validated error: 0.008057314749798879
[I 2025-10-27 23:22:15,352] Trial 1 finished with value: 0.07365868194019859 and parameters: {'lr': 0.0001784325070080417, 'num_epochs': 1, 'batch_size': 64, 'num_neighbors': 10, 'lr_decay': 0.9664030861479147}. Best is trial 1 with value: 0.07365868194019859.


...

Train wi info: {'gini': np.float64(0.5318939189491736), 'ess': np.float64(5139.410859868155), 'max_wi': np.float64(9.686144553157186), 'min_wi': np.float64(0.03906758043679743)}
actual reward: [0.08639666]
{'gini': np.float64(0.5291076997182486), 'ess': np.float64(3665.6947415644804), 'max_wi': np.float64(9.49802626945328), 'min_wi': np.float64(0.05051829570017257)}
Cross-validated error: 0.008508793611584187
[I 2025-10-27 23:34:07,858] Trial 18 finished with value: 0.0752473977356801 and parameters: {'lr': 0.0009746252643148638, 'num_epochs': 5, 'batch_size': 256, 'num_neighbors': 6, 'lr_decay': 0.9855017085376683}. Best is trial 18 with value: 0.0752473977356801.


Best trial: 19. Best value: 0.0775719: 100%|██████████| 20/20 [13:59<00:00, 41.97s/it]

Train wi info: {'gini': np.float64(0.8921142883907963), 'ess': np.float64(1376.812637891251), 'max_wi': np.float64(24.63087778020697), 'min_wi': np.float64(0.0002894263923630555)}
actual reward: [0.08591988]
{'gini': np.float64(0.882510728704491), 'ess': np.float64(1019.9244806439935), 'max_wi': np.float64(24.63087778020697), 'min_wi': np.float64(0.00028341453096947624)}
Cross-validated error: 0.009247601709633726
[I 2025-10-27 23:34:49,382] Trial 19 finished with value: 0.07757193181161719 and parameters: {'lr': 0.001462337473591599, 'num_epochs': 6, 'batch_size': 256, 'num_neighbors': 6, 'lr_decay': 0.9802496749954691}. Best is trial 19 with value: 0.07757193181161719.


{'gini': np.float64(0.9512420704631571), 'ess': np.float64(331.3872075421497), 'max_wi': np.float64(221.85069782106882), 'min_wi': np.float64(4.279189116368622e-06)}


,policy_rewards,ipw,reg_dm,conv_dm,conv_dr,conv_sndr,action_diff_to_real,action_delta,context_diff_to_real,context_delta
0,0.08610747,0.08790000,0.08802531,0.09188019,0.08858942,0.08858942,0.75692870,0.00000000,0.87627132,0.00000000
15000,0.08516441,0.08017263,0.08906158,0.09463238,0.07956430,0.07993675,0.82602002,0.24147136,0.93809648,0.12803130


### Policy with delta function

In [7]:
dataset_params = dict(
                    n_actions= 500,
                    n_users = 500,
                    emb_dim = 16,
                    # sigma = 0.1,
                    eps = 0.6, # this is the epsilon for the noise in the ground truth policy representation
                    ctr = 0.1
                    )

train_dataset = generate_dataset(dataset_params, seed=10000)

Random Item CTR: 0.07083863592474163
Optimal greedy CTR: 0.09999916436977967
Optimal Stochastic CTR: 0.0999493542444427
Our Initial CTR: 0.08557719469284641


In [8]:
# Run the optimization
df5, best_hyperparams_by_size = trainer_trial(num_runs, num_neighbors, num_rounds_list, train_dataset, batch_size, val_size=10000, n_trials=n_trials_for_optuna, prev_best_params=best_params_to_use)

# Show the performance metrics
df5[['policy_rewards', 'ipw', 'reg_dm', 'conv_dm', 'conv_dr', 'conv_sndr', 'action_diff_to_real', 'action_delta', 'context_diff_to_real', 'context_delta']]

{'gini': np.float64(0.44923948239866207), 'ess': np.float64(4832.412848990686), 'max_wi': np.float64(21.27427255219379), 'min_wi': np.float64(0.014521519317996934)}


[I 2025-10-27 23:36:00,165] A new study created in memory with name: no-name-f0d13181-70b5-4258-b7cb-4aa8f1d8a78a
Best trial: 0. Best value: 0.0880204:   5%|▌         | 1/20 [00:41<13:00, 41.08s/it]

Train wi info: {'gini': np.float64(0.9996097069643727), 'ess': np.float64(5.830156890001412), 'max_wi': np.float64(3630.058135095379), 'min_wi': np.float64(0.0)}
actual reward: [0.07890449]
{'gini': np.float64(0.9994245666139087), 'ess': np.float64(6.375448159220158), 'max_wi': np.float64(1614.386753291461), 'min_wi': np.float64(0.0)}
Cross-validated error: 0.014929009492012396
[I 2025-10-27 23:36:41,239] Trial 0 finished with value: 0.08802041750421653 and parameters: {'lr': 0.096, 'num_epochs': 5, 'batch_size': 64, 'num_neighbors': 8, 'lr_decay': 0.85}. Best is trial 0 with value: 0.08802041750421653.


Best trial: 0. Best value: 0.0880204:  10%|█         | 2/20 [01:25<12:51, 42.86s/it]

Train wi info: {'gini': np.float64(0.9998221742259609), 'ess': np.float64(2.0554121861970804), 'max_wi': np.float64(8030.121123700266), 'min_wi': np.float64(1.935631442710909e-34)}
actual reward: [0.07658656]
{'gini': np.float64(0.9995960337073696), 'ess': np.float64(4.427802787389191), 'max_wi': np.float64(2116.049297312971), 'min_wi': np.float64(1.935631442710909e-34)}
Cross-validated error: 0.007664841123861136
[I 2025-10-27 23:37:25,351] Trial 1 finished with value: 0.07221835978638602 and parameters: {'lr': 0.01101759311492387, 'num_epochs': 10, 'batch_size': 64, 'num_neighbors': 8, 'lr_decay': 0.9727821458933623}. Best is trial 0 with value: 0.08802041750421653.


...

Train wi info: {'gini': np.float64(0.99979825330962), 'ess': np.float64(3.254814397571667), 'max_wi': np.float64(7462.3827316229335), 'min_wi': np.float64(5.825886907571239e-22)}
actual reward: [0.06863549]
{'gini': np.float64(0.9998413283055675), 'ess': np.float64(1.6998012443673802), 'max_wi': np.float64(5981.249174477704), 'min_wi': np.float64(6.470961990118268e-22)}
Cross-validated error: 0.0037177673982042455
[I 2025-10-27 23:48:48,029] Trial 18 finished with value: 0.05353168241433952 and parameters: {'lr': 0.004753717233942495, 'num_epochs': 7, 'batch_size': 64, 'num_neighbors': 6, 'lr_decay': 0.8172081970678516}. Best is trial 0 with value: 0.08802041750421653.


Best trial: 0. Best value: 0.0880204: 100%|██████████| 20/20 [13:27<00:00, 40.38s/it]

Train wi info: {'gini': np.float64(0.4280820350415511), 'ess': np.float64(7907.601050502945), 'max_wi': np.float64(5.327344295792663), 'min_wi': np.float64(0.08180966994520852)}
actual reward: [0.08524725]
{'gini': np.float64(0.42059953857286125), 'ess': np.float64(5569.96297740022), 'max_wi': np.float64(4.974736160629101), 'min_wi': np.float64(0.08180966994520852)}
Cross-validated error: 0.008630729749377779
[I 2025-10-27 23:49:27,734] Trial 19 finished with value: 0.07562763067245258 and parameters: {'lr': 0.0013445150172097717, 'num_epochs': 5, 'batch_size': 512, 'num_neighbors': 8, 'lr_decay': 0.8559553230590546}. Best is trial 0 with value: 0.08802041750421653.


{'gini': np.float64(0.9997772353395038), 'ess': np.float64(2.2784141545297385), 'max_wi': np.float64(5667.301413335771), 'min_wi': np.float64(0.0)}


,policy_rewards,ipw,reg_dm,conv_dm,conv_dr,conv_sndr,action_diff_to_real,action_delta,context_diff_to_real,context_delta
0,0.08557719,0.08560000,0.08549812,0.08946431,0.08491724,0.08491724,0.82618217,0.00000000,0.99950468,0.00000000
15000,0.07726057,0.00000000,0.09523702,0.05551238,0.02801009,-0.04978863,1.84970676,1.68611378,1.44748668,0.75767119


In [9]:
dataset_params = dict(
                    n_actions= 500,
                    n_users = 500,
                    emb_dim = 16,
                    # sigma = 0.1,
                    eps = 0.6, # this is the epsilon for the noise in the ground truth policy representation
                    ctr = 0.1
                    )

train_dataset = generate_dataset(dataset_params, seed=20000)

Random Item CTR: 0.07042251854546815
Optimal greedy CTR: 0.09999934264692525
Optimal Stochastic CTR: 0.09996075464321043
Our Initial CTR: 0.08647580588501355


In [10]:
# Run the optimization
df6, best_hyperparams_by_size = trainer_trial(num_runs, num_neighbors, num_rounds_list, train_dataset, batch_size, val_size=10000, n_trials=n_trials_for_optuna, prev_best_params=best_params_to_use)

# Show the performance metrics
df6[['policy_rewards', 'ipw', 'reg_dm', 'conv_dm', 'conv_dr', 'conv_sndr', 'action_diff_to_real', 'action_delta', 'context_diff_to_real', 'context_delta']]

{'gini': np.float64(0.46252337713872943), 'ess': np.float64(4199.2626855415365), 'max_wi': np.float64(35.00371895533248), 'min_wi': np.float64(0.015165627151326161)}


[I 2025-10-27 23:50:35,917] A new study created in memory with name: no-name-e0614b86-a2cd-4ea9-aed5-a9cdae968b27
Best trial: 0. Best value: 0.0712219:   5%|▌         | 1/20 [00:41<13:08, 41.50s/it]

Train wi info: {'gini': np.float64(0.9994312030295652), 'ess': np.float64(9.401063447168319), 'max_wi': np.float64(2411.9660013041316), 'min_wi': np.float64(0.0)}
actual reward: [0.08270539]
{'gini': np.float64(0.998834130777689), 'ess': np.float64(12.449745870924417), 'max_wi': np.float64(2563.59741867418), 'min_wi': np.float64(0.0)}
Cross-validated error: 0.008157016065257488
[I 2025-10-27 23:51:17,412] Trial 0 finished with value: 0.07122186095641528 and parameters: {'lr': 0.096, 'num_epochs': 5, 'batch_size': 64, 'num_neighbors': 8, 'lr_decay': 0.85}. Best is trial 0 with value: 0.07122186095641528.


Best trial: 1. Best value: 0.0714719:  10%|█         | 2/20 [01:21<12:11, 40.64s/it]

Train wi info: {'gini': np.float64(0.1257473653317675), 'ess': np.float64(14086.590198829628), 'max_wi': np.float64(2.56468539917588), 'min_wi': np.float64(0.4508305572470686)}
actual reward: [0.0865818]
{'gini': np.float64(0.12299934376143441), 'ess': np.float64(9451.312340616245), 'max_wi': np.float64(2.7402104156322737), 'min_wi': np.float64(0.49551307094742053)}
Cross-validated error: 0.007471196584155263
[I 2025-10-27 23:51:57,449] Trial 1 finished with value: 0.07147194404105459 and parameters: {'lr': 0.0012321998583674654, 'num_epochs': 5, 'batch_size': 512, 'num_neighbors': 8, 'lr_decay': 0.8045851102528513}. Best is trial 1 with value: 0.07147194404105459.


...

Train wi info: {'gini': np.float64(0.9993640529649561), 'ess': np.float64(10.537726252969954), 'max_wi': np.float64(1168.645500627307), 'min_wi': np.float64(6.016430614358391e-21)}
actual reward: [0.08051569]
{'gini': np.float64(0.9988934204763997), 'ess': np.float64(11.046908740763518), 'max_wi': np.float64(1424.1011766179781), 'min_wi': np.float64(7.081151830313337e-21)}
Cross-validated error: 0.007531752584274878
[I 2025-10-28 00:03:34,776] Trial 18 finished with value: 0.07174998021052512 and parameters: {'lr': 0.003444957483594971, 'num_epochs': 9, 'batch_size': 64, 'num_neighbors': 10, 'lr_decay': 0.8968488577467927}. Best is trial 3 with value: 0.07997627282423043.


Best trial: 3. Best value: 0.0799763: 100%|██████████| 20/20 [13:38<00:00, 40.93s/it]

Train wi info: {'gini': np.float64(0.13375982388872948), 'ess': np.float64(13827.172150936609), 'max_wi': np.float64(3.2117052021458963), 'min_wi': np.float64(0.40741735250198163)}
actual reward: [0.08660467]
{'gini': np.float64(0.132022604738014), 'ess': np.float64(9276.472909661627), 'max_wi': np.float64(3.5697834348430355), 'min_wi': np.float64(0.447558851582022)}
Cross-validated error: 0.007522171641492355
[I 2025-10-28 00:04:14,469] Trial 19 finished with value: 0.07167480223314454 and parameters: {'lr': 0.0007491708504922513, 'num_epochs': 4, 'batch_size': 256, 'num_neighbors': 13, 'lr_decay': 0.950573235034753}. Best is trial 3 with value: 0.07997627282423043.


{'gini': np.float64(0.9878041884816501), 'ess': np.float64(93.75247677853959), 'max_wi': np.float64(262.31639900636765), 'min_wi': np.float64(6.034154546512548e-14)}


,policy_rewards,ipw,reg_dm,conv_dm,conv_dr,conv_sndr,action_diff_to_real,action_delta,context_diff_to_real,context_delta
0,0.08647581,0.08830000,0.08831187,0.08828296,0.08685068,0.08685068,0.88083979,0.00000000,0.74725465,0.00000000
15000,0.08341970,0.09461783,0.09015125,0.10100693,0.09713463,0.09684370,1.02117958,0.37188423,0.92463774,0.32452709


In [11]:
dataset_params = dict(
                    n_actions= 500,
                    n_users = 500,
                    emb_dim = 16,
                    # sigma = 0.1,
                    eps = 0.6, # this is the epsilon for the noise in the ground truth policy representation
                    ctr = 0.1
                    )

train_dataset = generate_dataset(dataset_params, seed=30000)

Random Item CTR: 0.07069350185865088
Optimal greedy CTR: 0.09999918303816259
Optimal Stochastic CTR: 0.0999509448932121
Our Initial CTR: 0.08653966603258505


In [12]:
# Run the optimization
df7, best_hyperparams_by_size = trainer_trial(num_runs, num_neighbors, num_rounds_list, train_dataset, batch_size, val_size=10000, n_trials=n_trials_for_optuna, prev_best_params=best_params_to_use)

# Show the performance metrics
df7[['policy_rewards', 'ipw', 'reg_dm', 'conv_dm', 'conv_dr', 'conv_sndr', 'action_diff_to_real', 'action_delta', 'context_diff_to_real', 'context_delta']]

{'gini': np.float64(0.46691230806930045), 'ess': np.float64(4435.710165485847), 'max_wi': np.float64(25.67323418791716), 'min_wi': np.float64(0.015833285225929968)}


[I 2025-10-28 00:05:22,516] A new study created in memory with name: no-name-5585ab91-e63c-4883-b5c9-c58aa40f79fe
Best trial: 0. Best value: 0.0684667:   5%|▌         | 1/20 [00:41<13:05, 41.36s/it]

Train wi info: {'gini': np.float64(0.9992690163389645), 'ess': np.float64(11.54661790260458), 'max_wi': np.float64(2110.4958967186294), 'min_wi': np.float64(0.0)}
actual reward: [0.0765924]
{'gini': np.float64(0.9992193085635824), 'ess': np.float64(8.583189658067637), 'max_wi': np.float64(2520.606918191326), 'min_wi': np.float64(0.0)}
Cross-validated error: 0.007327511236574311
[I 2025-10-28 00:06:03,871] Trial 0 finished with value: 0.06846671806879495 and parameters: {'lr': 0.096, 'num_epochs': 5, 'batch_size': 64, 'num_neighbors': 8, 'lr_decay': 0.85}. Best is trial 0 with value: 0.06846671806879495.


Best trial: 1. Best value: 0.073837:  10%|█         | 2/20 [01:20<12:02, 40.16s/it] 

Train wi info: {'gini': np.float64(0.027310923686200736), 'ess': np.float64(14954.46466633772), 'max_wi': np.float64(1.3390055342022473), 'min_wi': np.float64(0.8402084858569301)}
actual reward: [0.0865726]
{'gini': np.float64(0.026764246775215227), 'ess': np.float64(9971.683839404637), 'max_wi': np.float64(1.3299329635273056), 'min_wi': np.float64(0.8579692861930686)}
Cross-validated error: 0.008115159540973046
[I 2025-10-28 00:06:43,195] Trial 1 finished with value: 0.07383702561624221 and parameters: {'lr': 0.00045238085386773347, 'num_epochs': 2, 'batch_size': 256, 'num_neighbors': 15, 'lr_decay': 0.8670904426471295}. Best is trial 1 with value: 0.07383702561624221.


...

Train wi info: {'gini': np.float64(0.9995841280888977), 'ess': np.float64(3.9353100648258694), 'max_wi': np.float64(8284.861406059881), 'min_wi': np.float64(1.6654705220828945e-33)}
actual reward: [0.07983486]
{'gini': np.float64(0.9989287115028471), 'ess': np.float64(11.120027323340231), 'max_wi': np.float64(1335.3525780386265), 'min_wi': np.float64(1.6654705220828945e-33)}
Cross-validated error: 0.010218239179396859
[I 2025-10-28 00:18:26,574] Trial 18 finished with value: 0.08055358904798596 and parameters: {'lr': 0.021481255407433664, 'num_epochs': 4, 'batch_size': 128, 'num_neighbors': 10, 'lr_decay': 0.9133768116739862}. Best is trial 18 with value: 0.08055358904798596.


Best trial: 18. Best value: 0.0805536: 100%|██████████| 20/20 [13:44<00:00, 41.23s/it]

Train wi info: {'gini': np.float64(0.9943218377615222), 'ess': np.float64(53.892139439341456), 'max_wi': np.float64(1514.6693686189371), 'min_wi': np.float64(5.601599163940932e-29)}
actual reward: [0.07374614]
{'gini': np.float64(0.9955133023881012), 'ess': np.float64(26.46244506776238), 'max_wi': np.float64(1294.5221231175915), 'min_wi': np.float64(5.601599163940932e-29)}
Cross-validated error: 0.006241128218861144
[I 2025-10-28 00:19:07,026] Trial 19 finished with value: 0.06655811771799952 and parameters: {'lr': 0.017472763331314176, 'num_epochs': 3, 'batch_size': 128, 'num_neighbors': 10, 'lr_decay': 0.9699543382075912}. Best is trial 18 with value: 0.08055358904798596.


{'gini': np.float64(0.9973610174551267), 'ess': np.float64(15.827750037516562), 'max_wi': np.float64(1978.5128751704929), 'min_wi': np.float64(1.2503375789455513e-33)}


,policy_rewards,ipw,reg_dm,conv_dm,conv_dr,conv_sndr,action_diff_to_real,action_delta,context_diff_to_real,context_delta
0,0.08653967,0.08730000,0.08727474,0.08935798,0.08737408,0.08737408,0.82469903,0.00000000,0.72168239,0.00000000
15000,0.07620169,0.02471816,0.08501919,0.08134475,0.01775477,0.02246849,1.02339782,0.55808987,1.05525800,0.57137736


In [13]:
dataset_params = dict(
                    n_actions= 500,
                    n_users = 500,
                    emb_dim = 16,
                    # sigma = 0.1,
                    eps = 0.6, # this is the epsilon for the noise in the ground truth policy representation
                    ctr = 0.1
                    )

train_dataset = generate_dataset(dataset_params, seed=40000)

Random Item CTR: 0.07053370144999074
Optimal greedy CTR: 0.09999936716169436
Optimal Stochastic CTR: 0.09995563088920843
Our Initial CTR: 0.08622184481781218


In [14]:
# Run the optimization
df8, best_hyperparams_by_size = trainer_trial(num_runs, num_neighbors, num_rounds_list, train_dataset, batch_size, val_size=10000, n_trials=n_trials_for_optuna, prev_best_params=best_params_to_use)

# Show the performance metrics
df8[['policy_rewards', 'ipw', 'reg_dm', 'conv_dm', 'conv_dr', 'conv_sndr', 'action_diff_to_real', 'action_delta', 'context_diff_to_real', 'context_delta']]

{'gini': np.float64(0.4765880277297409), 'ess': np.float64(4297.400342742042), 'max_wi': np.float64(23.98424839857256), 'min_wi': np.float64(0.008677272580760257)}


[I 2025-10-28 00:20:15,279] A new study created in memory with name: no-name-cd3a4713-a607-496b-8326-60b559875358
Best trial: 0. Best value: 0.071175:   5%|▌         | 1/20 [00:41<13:16, 41.94s/it]

Train wi info: {'gini': np.float64(0.9994425534628644), 'ess': np.float64(8.39211110682246), 'max_wi': np.float64(6455.883621370989), 'min_wi': np.float64(0.0)}
actual reward: [0.0781076]
{'gini': np.float64(0.9995481682247254), 'ess': np.float64(4.851856347715597), 'max_wi': np.float64(3550.1441402641103), 'min_wi': np.float64(0.0)}
Cross-validated error: 0.006006053180833118
[I 2025-10-28 00:20:57,218] Trial 0 finished with value: 0.07117500293424119 and parameters: {'lr': 0.096, 'num_epochs': 5, 'batch_size': 64, 'num_neighbors': 8, 'lr_decay': 0.85}. Best is trial 0 with value: 0.07117500293424119.


Best trial: 0. Best value: 0.071175:  10%|█         | 2/20 [01:22<12:20, 41.12s/it]

Train wi info: {'gini': np.float64(0.9955753189931694), 'ess': np.float64(20.342612678521512), 'max_wi': np.float64(626.6510327448123), 'min_wi': np.float64(3.424673251662064e-33)}
actual reward: [0.07547815]
{'gini': np.float64(0.9978620604201811), 'ess': np.float64(9.298761577717519), 'max_wi': np.float64(1952.6189195954032), 'min_wi': np.float64(3.0333617777215167e-33)}
Cross-validated error: 0.006764925460367622
[I 2025-10-28 00:21:37,769] Trial 1 finished with value: 0.0686914786145342 and parameters: {'lr': 0.03051639043783386, 'num_epochs': 3, 'batch_size': 256, 'num_neighbors': 11, 'lr_decay': 0.887472700875917}. Best is trial 0 with value: 0.07117500293424119.


...

Train wi info: {'gini': np.float64(0.9760187543450577), 'ess': np.float64(286.2071625090533), 'max_wi': np.float64(401.27750549169195), 'min_wi': np.float64(1.108212178095789e-08)}
actual reward: [0.07910612]
{'gini': np.float64(0.9753833718344049), 'ess': np.float64(184.1559652007926), 'max_wi': np.float64(416.1593594126596), 'min_wi': np.float64(2.005611101825991e-08)}
Cross-validated error: 0.008771668768240292
[I 2025-10-28 00:33:18,965] Trial 18 finished with value: 0.07605118699761479 and parameters: {'lr': 0.0007838291241515988, 'num_epochs': 6, 'batch_size': 64, 'num_neighbors': 5, 'lr_decay': 0.969057498988737}. Best is trial 17 with value: 0.08051659731920405.


Best trial: 17. Best value: 0.0805166: 100%|██████████| 20/20 [13:45<00:00, 41.26s/it]

Train wi info: {'gini': np.float64(0.13459718692174996), 'ess': np.float64(13562.396871452263), 'max_wi': np.float64(3.4197225701756357), 'min_wi': np.float64(0.4012525586365764)}
actual reward: [0.08640222]
{'gini': np.float64(0.13669370559506883), 'ess': np.float64(9062.284363197758), 'max_wi': np.float64(3.4197225701756357), 'min_wi': np.float64(0.4503985062471784)}
Cross-validated error: 0.00810147130095046
[I 2025-10-28 00:34:00,413] Trial 19 finished with value: 0.07376464151615597 and parameters: {'lr': 0.00022140417097077505, 'num_epochs': 7, 'batch_size': 128, 'num_neighbors': 3, 'lr_decay': 0.9296482058187899}. Best is trial 17 with value: 0.08051659731920405.


{'gini': np.float64(0.6875407660566112), 'ess': np.float64(866.7547908092289), 'max_wi': np.float64(125.17520581288665), 'min_wi': np.float64(0.0033300205141579753)}


,policy_rewards,ipw,reg_dm,conv_dm,conv_dr,conv_sndr,action_diff_to_real,action_delta,context_diff_to_real,context_delta
0,0.08622184,0.08840000,0.08844068,0.09405586,0.08962762,0.08962762,0.92210476,0.00000000,0.83772226,0.00000000
15000,0.08710871,0.07826932,0.08687713,0.09178597,0.07893424,0.07893299,0.92695644,0.09435642,0.84337890,0.04041727


In [15]:
dataset_params = dict(
                    n_actions= 500,
                    n_users = 500,
                    emb_dim = 16,
                    # sigma = 0.1,
                    eps = 0.6, # this is the epsilon for the noise in the ground truth policy representation
                    ctr = 0.1
                    )

train_dataset = generate_dataset(dataset_params, seed=50000)

Random Item CTR: 0.0705882181025533
Optimal greedy CTR: 0.09999934164533562
Optimal Stochastic CTR: 0.09995498601895662
Our Initial CTR: 0.08647501952799874


In [16]:
# Run the optimization
df9, best_hyperparams_by_size = trainer_trial(num_runs, num_neighbors, num_rounds_list, train_dataset, batch_size, val_size=10000, n_trials=n_trials_for_optuna, prev_best_params=best_params_to_use)

# Show the performance metrics
df9[['policy_rewards', 'ipw', 'reg_dm', 'conv_dm', 'conv_dr', 'conv_sndr', 'action_diff_to_real', 'action_delta', 'context_diff_to_real', 'context_delta']]

{'gini': np.float64(0.48346307169006664), 'ess': np.float64(4282.643856377782), 'max_wi': np.float64(21.56110254536132), 'min_wi': np.float64(0.010593760814322993)}


[I 2025-10-28 00:35:08,682] A new study created in memory with name: no-name-a60aae64-b943-4cb3-b524-e468919fbb37
Best trial: 0. Best value: 0.0757378:   5%|▌         | 1/20 [00:41<13:10, 41.60s/it]

Train wi info: {'gini': np.float64(0.9992959879929818), 'ess': np.float64(11.584828777106043), 'max_wi': np.float64(4139.250556300044), 'min_wi': np.float64(0.0)}
actual reward: [0.07880265]
{'gini': np.float64(0.9994971649188609), 'ess': np.float64(5.433174965276248), 'max_wi': np.float64(3439.698557222427), 'min_wi': np.float64(0.0)}
Cross-validated error: 0.006809658573854237
[I 2025-10-28 00:35:50,283] Trial 0 finished with value: 0.07573778353316493 and parameters: {'lr': 0.096, 'num_epochs': 5, 'batch_size': 64, 'num_neighbors': 8, 'lr_decay': 0.85}. Best is trial 0 with value: 0.07573778353316493.


Best trial: 0. Best value: 0.0757378:  10%|█         | 2/20 [01:20<12:03, 40.18s/it]

Train wi info: {'gini': np.float64(0.3669867948930161), 'ess': np.float64(8058.104836130282), 'max_wi': np.float64(9.36129950207573), 'min_wi': np.float64(0.08691758279234292)}
actual reward: [0.08721718]
{'gini': np.float64(0.3639490799022447), 'ess': np.float64(5581.674117391761), 'max_wi': np.float64(9.36129950207573), 'min_wi': np.float64(0.09693829087066094)}
Cross-validated error: 0.008202672137022157
[I 2025-10-28 00:36:29,465] Trial 1 finished with value: 0.07416106894605753 and parameters: {'lr': 0.004284887885089372, 'num_epochs': 1, 'batch_size': 256, 'num_neighbors': 6, 'lr_decay': 0.9384921659739343}. Best is trial 0 with value: 0.07573778353316493.


...

Train wi info: {'gini': np.float64(0.9995665905650811), 'ess': np.float64(7.2599986968445105), 'max_wi': np.float64(1953.9606705833335), 'min_wi': np.float64(6.915787852514309e-29)}
actual reward: [0.08130133]
{'gini': np.float64(0.9995333412293267), 'ess': np.float64(5.0904239773481175), 'max_wi': np.float64(2921.146967368231), 'min_wi': np.float64(9.237573833884467e-29)}
Cross-validated error: 0.005944900535702796
[I 2025-10-28 00:48:17,383] Trial 18 finished with value: 0.06520743906960992 and parameters: {'lr': 0.010237900630086916, 'num_epochs': 6, 'batch_size': 128, 'num_neighbors': 8, 'lr_decay': 0.9389372936027914}. Best is trial 8 with value: 0.1255423126235649.


Best trial: 8. Best value: 0.125542: 100%|██████████| 20/20 [13:50<00:00, 41.50s/it]

Train wi info: {'gini': np.float64(0.9555948736661486), 'ess': np.float64(679.4494520854925), 'max_wi': np.float64(66.74480736233829), 'min_wi': np.float64(6.305202236406696e-08)}
actual reward: [0.08929322]
{'gini': np.float64(0.9508456492529058), 'ess': np.float64(484.1616772890529), 'max_wi': np.float64(83.04955803682469), 'min_wi': np.float64(6.305202236406696e-08)}
Cross-validated error: 0.00720877371123147
[I 2025-10-28 00:48:58,761] Trial 19 finished with value: 0.07055214562694888 and parameters: {'lr': 0.00158959421333582, 'num_epochs': 9, 'batch_size': 256, 'num_neighbors': 5, 'lr_decay': 0.9816251741348216}. Best is trial 8 with value: 0.1255423126235649.


{'gini': np.float64(0.9996274160207251), 'ess': np.float64(4.076337164453286), 'max_wi': np.float64(2820.2154654927654), 'min_wi': np.float64(3.673717188363759e-35)}


,policy_rewards,ipw,reg_dm,conv_dm,conv_dr,conv_sndr,action_diff_to_real,action_delta,context_diff_to_real,context_delta
0,0.08647502,0.08530000,0.08531822,0.08859722,0.08369196,0.08369196,0.80232812,0.00000000,0.84032376,0.00000000
15000,0.07913758,0.50447760,0.09883096,0.08630665,0.54980022,0.50045389,1.12744744,0.75899290,1.15723686,0.53516479


In [17]:
df4[['policy_rewards', 'ipw', 'reg_dm', 'conv_dm', 'conv_dr', 'conv_sndr', 'action_diff_to_real', 'action_delta', 'context_diff_to_real', 'context_delta']]

,policy_rewards,ipw,reg_dm,conv_dm,conv_dr,conv_sndr,action_diff_to_real,action_delta,context_diff_to_real,context_delta
0,0.08610747,0.08790000,0.08802531,0.09188019,0.08858942,0.08858942,0.75692870,0.00000000,0.87627132,0.00000000
15000,0.08516441,0.08017263,0.08906158,0.09463238,0.07956430,0.07993675,0.82602002,0.24147136,0.93809648,0.12803130


### Poicy Via argmax(r_hat - error_hat) through cross validation

In [18]:
df4[['policy_rewards', 'ipw', 'reg_dm', 'conv_dm', 'conv_dr', 'conv_sndr', 'action_diff_to_real', 'action_delta', 'context_diff_to_real', 'context_delta']]

,policy_rewards,ipw,reg_dm,conv_dm,conv_dr,conv_sndr,action_diff_to_real,action_delta,context_diff_to_real,context_delta
0,0.08610747,0.08790000,0.08802531,0.09188019,0.08858942,0.08858942,0.75692870,0.00000000,0.87627132,0.00000000
15000,0.08516441,0.08017263,0.08906158,0.09463238,0.07956430,0.07993675,0.82602002,0.24147136,0.93809648,0.12803130


### Policy Via using actual policy value

In [19]:
# Show the performance metrics
df4[['policy_rewards', 'ipw', 'reg_dm', 'conv_dm', 'conv_dr', 'conv_sndr', 'action_diff_to_real', 'action_delta', 'context_diff_to_real', 'context_delta']]


,policy_rewards,ipw,reg_dm,conv_dm,conv_dr,conv_sndr,action_diff_to_real,action_delta,context_diff_to_real,context_delta
0,0.08610747,0.08790000,0.08802531,0.09188019,0.08858942,0.08858942,0.75692870,0.00000000,0.87627132,0.00000000
15000,0.08516441,0.08017263,0.08906158,0.09463238,0.07956430,0.07993675,0.82602002,0.24147136,0.93809648,0.12803130
